### Imports

In [9]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.tree import DecisionTreeClassifier

### Helper functions

In [6]:
def compute_error(y, y_pred, w_i):
    """
    Calculate the error rate of a weak classifier m
    
    :param y: actual target value
    :param y_pred: predicted value by weak classifier
    :param w_i: individual weights for each observationn
    :return: Error rate
    
    Note that all arrays should be the same length
    """
    return (sum(w_i * (np.not_equal(y, y_pred)).astype(int))) / sum(w_i)

In [7]:
def compute_alpha(error):
    """
    Calculate the weight(α) of a weak classifier m in the majority vote of the final classifier.
    This is called alpha(α) in chapter 10.1 of The Elements of Statistics Learning
    : param error: error rate from weak classifier m
    : return: Weight/alpha(α)
    """
    return np.log((1-error)/error)
    

In [8]:
def update_weights(w_i, alpha, y, y_pred):
    """
    Update individual weights w_i after a boosting iteration.
    :param w_i: individual weights for each observation
    :param y: actual target value
    :param y_pred: predicted value by weak classifier
    :param alpha: weight of weak classifier used to estimate y_pred
    """
    return w_i * np.exp(alpha * (np.not_equal(y, y_pred)).astype(int))

### Define AdaBoost Class

In [44]:
class AdaBoost:
    
    def __init__(self):
        self.alphas = []
        self.G_M = []
        self.M = None
        self.training_errors = []
        self.prediction_errors = []
        
        
    def fit(self, X, y, M=100):
        """
        Fit Models
        :param X: independent variables - array-like matrix
        :param y: target variable - array-like vector
        :param M: number of boosting rounds. Default is 100 - integer
        """
        # Clear before calling
        self.alphas = []
        self.training_errors = []
        self.M = M
        
        # Iterate over M weak classifiers
        for m in range(0, M):
            
            # Set weights for current boosting iteration
            if m==0:
                
                w_i = np.ones(len(y)) * 1 / len(y) # At m=0, weights are all the same and equal to 1/N
                
            else:
                
                # (d) Update w_i
                w_i = update_weights(w_i, alpha=alpha_m, y=y, y_pred=y_pred)
                
#             print(w_i)
            
            
            # (a) Fit weak classifier and predict labels
            G_m = DecisionTreeClassifier(max_depth=1) #Stump: Two terminal-node classification tree
            G_m.fit(X, y, sample_weight=w_i)
            y_pred = G_m.predict(X)
            
            self.G_M.append(G_m) # Save to list of weak classifiers
            
            
            # (b) Compute Error
            error_m = compute_error(y, y_pred, w_i)
            self.training_errors.append(error_m)
#             print(error_m)
            
            # (c) Compute Alpha
            alpha_m = compute_alpha(error_m)
            self.alphas.append(alpha_m)
#             print(alpha_m)
            
        assert len(self.G_M) == len(self.alphas)
        
    
    def predict(self, X):
        """
        Predict using fitted model.
        
        :param X: independent variables - array-like
        """
        # Initialise dataframe with weak predictions for each observations
        weak_preds = pd.DataFrame(index= range(len(X)), columns= range(self.M))
        
        # Predict class label for each weak classifier, weighted by alpha_m
        for m in range(self.M):
            y_pred_m = self.G_M[m].predict(X) * self.alphas[m]
            weak_preds.iloc[:,m] = y_pred_m
            
        # Calculate final predictions
        y_pred = (1 * np.sign(weak_preds.T.sum())).astype(int)
        
        return y_pred

In [45]:
# Dataset
df = pd.read_csv('spambase.data', header=None)

In [46]:
# Column Names
names = pd.read_csv('spambase.names', sep=':', skiprows=range(0, 33), header=None)

In [47]:
col_names = list(names[0])

In [48]:
col_names.append('Spam')

In [49]:
# Rename df columns
df.columns = col_names

In [50]:
# Convert classes in target variable to {-1, 1}
df['Spam'] = df['Spam'] * 2 - 1

In [51]:
# Train - test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = 'Spam').values, df['Spam'].values, train_size = 3065, random_state = 2) 

In [52]:
# Fit model
ab = AdaBoost()
ab.fit(X_train, y_train, M = 400)

In [53]:
# Predict on test set
y_pred = ab.predict(X_test)

In [54]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9440104166666666

### Comparison with Scikit-Learn

In [55]:
from sklearn.ensemble import AdaBoostClassifier

In [57]:
model = AdaBoostClassifier(n_estimators=400)
model.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=400)

In [58]:
y_pred = model.predict(X_test)

In [59]:
accuracy_score(y_test, y_pred)

0.931640625